In [ ]:
# Import python packages
import streamlit as st
import numpy as np
import pandas as pd
from snowflake.snowpark.functions import(
col, count, avg, max as max_, min as min_, dateadd, current_date, lit, sum as sum_, 
coalesce, datediff, any_value, when, array_unique_agg, array_size, regexp_replace, iff, nullifzero
)
from snowflake.ml.feature_store import FeatureStore, Entity, FeatureView, CreationMode

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Data preparation

#### Call quality

In [ ]:
user_features_call_quality_df = session.sql("""
SELECT 
    USER_ID_HEX,
    CAST(COUNT_IF(COALESCE(num_bad_mos_periods, 0) > 0) AS FLOAT) AS calls_with_bad_mos,
    CAST(AVG(computed_mos) AS FLOAT) AS average_mos,
    CAST(MAX(RTP_SETUP_TIME) AS FLOAT) AS max_rtp_setup_time,
    CAST(COUNT_IF(CASE WHEN call_date >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN COALESCE(num_bad_mos_periods, 0) > 0 ELSE FALSE END) AS FLOAT) AS calls_with_bad_mos_7d,
    CAST(AVG(CASE WHEN call_date >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN computed_mos END) AS FLOAT) AS average_mos_7d,
    CAST(MAX(CASE WHEN call_date >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN RTP_SETUP_TIME END) AS FLOAT) AS max_rtp_setup_time_7d
FROM dev.public.legacy_call_end
WHERE call_date <= CURRENT_DATE()
    AND USER_ID_HEX != '000-00-000-000000000'
GROUP BY USER_ID_HEX
""")
user_features_call_quality_df.show()

#### Call rating

In [ ]:
user_features_call_rating_df =session.sql("""
SELECT 
    user_id_hex,
    CAST(COUNT(call_rating) AS FLOAT) AS call_rating_count,
    CAST(AVG(call_rating) AS FLOAT) AS avg_call_rating,
    CAST(MAX(call_rating) AS FLOAT) AS max_call_rating,
    CAST(MIN(call_rating) AS FLOAT) AS min_call_rating,
    CAST(COUNT(CASE WHEN date_utc >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN call_rating END) AS FLOAT) AS call_rating_count_7d,
    CAST(AVG(CASE WHEN date_utc >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN call_rating END) AS FLOAT) AS avg_call_rating_7d,
    CAST(MAX(CASE WHEN date_utc >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN call_rating END) AS FLOAT) AS max_call_rating_7d,
    CAST(MIN(CASE WHEN date_utc >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN call_rating END) AS FLOAT) AS min_call_rating_7d
FROM dev.public.call_ratings_combined_sources
WHERE date_utc <= CURRENT_DATE()
    AND call_rating > 0
    AND user_id_hex != '000-00-000-000000000'
GROUP BY user_id_hex
""")
user_features_call_rating_df.show()

#### Data usage

In [ ]:
user_features_data_usage_df = session.sql("""
SELECT 
    up.user_id_hex,
    SUM(c.mb_usage) AS data_usage_mb,
    SUM(CASE WHEN c.date_utc >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN c.mb_usage ELSE 0 END) AS data_usage_mb_7d
FROM dev.public.cost_user_daily_tmobile_cost c
JOIN dev.public.user_profiles up ON c.username = up.latest_username
WHERE c.date_utc <= CURRENT_DATE()
GROUP BY up.user_id_hex
""")
user_features_data_usage_df.show()

#### Session

In [ ]:
user_features_sessions_df = session.sql("""
SELECT 
    up.user_id_hex,
    CAST(SUM(m.time_in_app_mins_per_day) AS FLOAT) AS time_in_app_mins,
    CAST(DATEDIFF(day, ANY_VALUE(up.registered_at), CURRENT_DATE()) AS FLOAT) AS tenure_days,
    CAST(SUM(m.num_sessions) AS FLOAT) AS session_count,
    CAST(SUM(CASE WHEN m.date_utc >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN m.time_in_app_mins_per_day ELSE 0 END) AS FLOAT) AS time_in_app_mins_7d,
    CAST(SUM(CASE WHEN m.date_utc >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN m.num_sessions ELSE 0 END) AS FLOAT) AS session_count_7d
FROM dev.public.metrics_daily_userlevel_app_time_sessions m
JOIN dev.public.user_profiles up ON m.username = up.latest_username
WHERE m.date_utc <= CURRENT_DATE()
GROUP BY up.user_id_hex
""")
user_features_sessions_df.show()

#### NPS ratings

In [ ]:
user_features_nps_rating_df = session.sql("""
SELECT 
    user_id_hex,
    CAST(COUNT(*) AS FLOAT) AS nps_count,
    CAST(AVG(score) AS FLOAT) AS nps_avg_rating,
    CAST(MAX(score) AS FLOAT) AS nps_max_rating,
    CAST(COUNT(CASE WHEN date_utc >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN 1 END) AS FLOAT) AS nps_count_7d,
    CAST(AVG(CASE WHEN date_utc >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN score END) AS FLOAT) AS nps_avg_rating_7d,
    CAST(MAX(CASE WHEN date_utc >= CURRENT_DATE() - INTERVAL '7 DAYS' THEN score END) AS FLOAT) AS nps_max_rating_7d
FROM dev.public.nps_combined_sources
WHERE date_utc <= CURRENT_DATE()
    AND user_id_hex != '000-00-000-000000000'
GROUP BY user_id_hex
""")
user_features_nps_rating_df.show()

## Feature store

#### Create FS

In [ ]:
# fs = FeatureStore(
#     session=session,
#     database="dev",
#     name="user_activity_feature_store",
#     default_warehouse="ds_wh_medium",
#     creation_mode=CreationMode.CREATE_IF_NOT_EXIST
# )

#### Connect to FS

In [ ]:
fs = FeatureStore(
    session=session,
    database="dev",
    name="user_activity_feature_store",
    default_warehouse="ds_wh_medium"
)

#### Create and register entities

In [ ]:
# entity = Entity(
#     name="user",
#     join_keys=["user_id_hex"],
#     desc="user entity"
# )
# fs.register_entity(entity)

#### Get existing entities

In [ ]:
entity = fs.get_entity("user")

#### Create and register feature views

In [ ]:
# Call quality
user_features_call_quality_fv = FeatureView(
    name="user_features_call_quality",
    entities=[entity],
    feature_df=user_features_call_quality_df,
    refresh_freq="24 hours",
    desc="features about user call quality"
)

fs.register_feature_view(
    feature_view=user_features_call_quality_fv,
    version="1"
)

In [ ]:
# Call rating
user_features_call_rating_fv = FeatureView(
    name="user_features_call_rating",
    entities=[entity],
    feature_df=user_features_call_rating_df,
    refresh_freq="24 hours",
    desc="features about user call rating"
)

fs.register_feature_view(
    feature_view=user_features_call_rating_fv,
    version="1"
)

In [ ]:
# Data usage
user_features_data_usage_fv = FeatureView(
    name="user_features_data_usage",
    entities=[entity],
    feature_df=user_features_data_usage_df,
    refresh_freq="24 hours",
    desc="features about user data usage"
)

fs.register_feature_view(
    feature_view=user_features_data_usage_fv,
    version="1"
)

In [ ]:
# User sessions
user_features_sessions_fv = FeatureView(
    name="user_features_sessions",
    entities=[entity],
    feature_df=user_features_sessions_df,
    refresh_freq="24 hours",
    desc="features about user sessions"
)

fs.register_feature_view(
    feature_view=user_features_sessions_fv,
    version="1"
)

In [ ]:
# NPS Rating
user_features_nps_rating_fv = FeatureView(
    name="user_features_nps_rating",
    entities=[entity],
    feature_df=user_features_nps_rating_df,
    refresh_freq="24 hours",
    desc="features about user NPS Rating"
)

fs.register_feature_view(
    feature_view=user_features_nps_rating_fv,
    version="1"
)

## Model training

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from snowflake.ml.experiment import ExperimentTracking

exp = ExperimentTracking(session=session)
exp.set_experiment("Baseline_user_activity_forecastint_models")

#### Spine df

In [ ]:
spine_df= session.sql("""
SELECT
    up.user_id_hex,
    sum(iff(m.time_in_app_mins_per_day > 1, 1, 0)) as active_days_in_week
FROM dev.public.metrics_daily_userlevel_app_time_sessions m
JOIN dev.public.user_profiles up ON m.username = up.latest_username
WHERE m.date_utc >= dateadd('day', -14, current_date())
GROUP by up.user_id_hex
""")
spine_df.show()

#### Get training dataset from FS

In [ ]:
user_features_call_quality=fs.get_feature_view(name="user_features_call_quality", version="1")
user_features_call_rating=fs.get_feature_view(name="user_features_call_rating", version="1")

df = fs.generate_training_set(
    spine_df=spine_df,
    features=[user_features_call_quality, user_features_call_rating],
    spine_label_cols="active_days_in_week"
)
df = df.to_pandas()
df.head()

In [ ]:
# Function to plot variable distributions
def plot_variable_distributions(dataframe):
    """Plot histograms for all numeric features"""
    # Get numeric columns
    numeric_columns = dataframe.select_dtypes(include=['number']).columns
    
    # Calculate grid size
    num_features = len(numeric_columns)
    num_cols = 3
    num_rows = (num_features + num_cols - 1) // num_cols
    
    # Create subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 4))
    axes = axes.flatten()
    
    # Plot each feature
    for i, column_name in enumerate(numeric_columns):
        axes[i].hist(dataframe[column_name].dropna(), bins=30, 
                    color='skyblue', edgecolor='black', alpha=0.7)
        axes[i].set_title(column_name, fontsize=10)
        axes[i].set_xlabel(column_name)
        axes[i].set_ylabel('Frequency')
        axes[i].grid(alpha=0.3)
    
    # Hide empty subplots
    for i in range(num_features, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle("Variable Distributions", fontsize=15, y=1.00)
    plt.tight_layout()
    plt.show()

# Call the function to plot distributions
plot_variable_distributions(df)

#### Feature Distribution

In [ ]:
# Function to plot correlation matrix
def plot_correlation_matrix(dataframe):
    """Plot heatmap showing correlations between features"""
    # Select only numeric columns
    numeric_data = dataframe.select_dtypes(include=['number'])
    
    # Calculate correlation matrix
    correlation_matrix = numeric_data.corr()
    
    # Create heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Feature Correlation Matrix', fontsize=16, pad=20)
    plt.tight_layout()
    plt.show()
    
    # Show correlation with target variable
    print("\nCorrelation with Target (ACTIVE_DAYS_IN_WEEK):")
    print("=" * 60)
    target_correlation = correlation_matrix['ACTIVE_DAYS_IN_WEEK'].sort_values(ascending=False)
    for feature, corr_value in target_correlation.items():
        print(f"{feature}: {corr_value:.4f}")
    print("=" * 60)

# Call the function to plot correlation matrix
plot_correlation_matrix(df)

#### Correlation Matrix

In [ ]:
# Configure pandas to show all columns
pd.set_option('display.max_columns', None)

# Function to explore the dataset
def explore_data(dataframe):
    """Display basic information about the dataset"""
    print("=" * 60)
    print("DATASET EXPLORATION")
    print("=" * 60)
    
    print("\nFirst few rows of the dataset:")
    print(dataframe.head())
    
    print("\nDataset Shape:")
    print(f"Rows: {dataframe.shape[0]}, Columns: {dataframe.shape[1]}")
    
    print("\nData Types of Each Column:")
    print(dataframe.dtypes)
    
    print("\nStatistics of the Numerical Columns:")
    print(dataframe.describe())
    
    print("\nMissing Values in Each Column:")
    print(dataframe.isnull().sum())
    
    print("\nTarget Variable (ACTIVE_DAYS_IN_WEEK) Distribution:")
    print(dataframe['ACTIVE_DAYS_IN_WEEK'].describe())
    print("=" * 60)

# Call the function to explore data
explore_data(df)

#### Dataset Overview

## Exploratory Data Analysis (EDA)

In [ ]:
# Separate features (X) and target (y)
X = df.drop(['USER_ID_HEX', 'ACTIVE_DAYS_IN_WEEK'], axis=1)
y = df['ACTIVE_DAYS_IN_WEEK']

print("=" * 60)
print("INITIAL FEATURE SET")
print("=" * 60)
print(f"Total features: {X.shape[1]}")
print(f"Total samples: {X.shape[0]}")
print(f"\nFeature names:")
for i, feature_name in enumerate(X.columns, 1):
    print(f"  {i}. {feature_name}")
print("=" * 60)

#### Prepare Features and Target

#### Split training and testing dataset

In [ ]:
# Split data into training and testing sets
print("=" * 60)
print("SPLITTING DATA INTO TRAIN AND TEST SETS")
print("=" * 60)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set: {X_train.shape[0]} samples, {X_train.shape[1]} features")
print(f"Test set: {X_test.shape[0]} samples, {X_test.shape[1]} features")
print("=" * 60)

In [ ]:
# Final feature set summary after feature selection
print("=" * 60)
print("FINAL FEATURE SET AFTER SELECTION")
print("=" * 60)
print(f"Total features selected: {X.shape[1]}")
print(f"\nSelected features for modeling:")
for i, feature_name in enumerate(X.columns, 1):
    print(f"  {i}. {feature_name}")
print("=" * 60)

#### Update X with selected features

In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Function to remove low variance features
def remove_low_variance_features(dataframe, variance_threshold=0.01):
    """Remove features that have very low variance (almost constant values)"""
    
    print("=" * 60)
    print("REMOVING LOW VARIANCE FEATURES")
    print("=" * 60)
    
    # Create variance selector
    variance_selector = VarianceThreshold(threshold=variance_threshold)
    variance_selector.fit(dataframe)
    
    # Get feature names that pass the threshold
    selected_features = dataframe.columns[variance_selector.get_support()].tolist()
    removed_features = dataframe.columns[~variance_selector.get_support()].tolist()
    
    # Display results
    print(f"Variance threshold: {variance_threshold}")
    print(f"Total features to remove: {len(removed_features)}")
    
    if removed_features:
        print("\nFeatures being removed (low variance):")
        for feature in removed_features:
            feature_variance = dataframe[feature].var()
            print(f"  - {feature} (variance: {feature_variance:.6f})")
    else:
        print("No low variance features found!")
    
    # Create dataframe with selected features
    dataframe_after_removal = dataframe[selected_features]
    
    print(f"\nFeatures before: {dataframe.shape[1]}")
    print(f"Features after: {dataframe_after_removal.shape[1]}")
    print("=" * 60)
    
    return dataframe_after_removal, removed_features

# Step 2: Remove low variance features from X
X = remove_low_variance_features(X, variance_threshold=0.01)[0]

#### Variance Threshold Feature Selection

In [ ]:
# Function to remove highly correlated features
def remove_correlated_features(dataframe, correlation_threshold=0.9):
    """Remove features that are highly correlated with each other"""
    
    print("=" * 60)
    print("REMOVING HIGHLY CORRELATED FEATURES")
    print("=" * 60)
    
    # Calculate correlation matrix
    correlation_matrix = dataframe.corr().abs()
    
    # Select upper triangle of correlation matrix
    upper_triangle = correlation_matrix.where(
        np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
    )
    
    # Find features with correlation greater than threshold
    features_to_drop = [column for column in upper_triangle.columns 
                       if any(upper_triangle[column] > correlation_threshold)]
    
    # Display results
    print(f"Correlation threshold: {correlation_threshold}")
    print(f"Total features to remove: {len(features_to_drop)}")
    
    if features_to_drop:
        print("\nFeatures being removed:")
        for feature in features_to_drop:
            print(f"  - {feature}")
    else:
        print("No highly correlated features found!")
    
    # Remove correlated features
    dataframe_after_removal = dataframe.drop(columns=features_to_drop)
    
    print(f"\nFeatures before: {dataframe.shape[1]}")
    print(f"Features after: {dataframe_after_removal.shape[1]}")
    print("=" * 60)
    
    return dataframe_after_removal, features_to_drop

# Step 1: Remove highly correlated features from X
X = remove_correlated_features(X, correlation_threshold=0.9)[0]

#### Remove Highly Correlated Features

## Feature Selection

#### Preprocessing

In [ ]:
numerical_cols = X.select_dtypes(include=['number']).columns
preprocess=ColumnTransformer([
    ("num", StandardScaler(), numerical_cols)
])

#### Base training

In [ ]:
baseline_models = [
    # Linear models
    ("LinearRegression", LinearRegression(), {}),
    ("Ridge", Ridge(), {'model__alpha': [0.1, 1.0, 10.0, 100.0]}),
    ("Lasso", Lasso(), {'model__alpha': [0.1, 1.0, 10.0, 100.0]}),
    
    # Random Forest variations
    ("RandomForest", RandomForestRegressor(), {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [5, 10, 15, None],
        'model__min_samples_split': [2, 5, 10]
    }),
    
    # Gradient Boosting
    ("GradientBoosting", GradientBoostingRegressor(), {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 7]
    }),
    
    # XGBoost
    ("XGBoost", XGBRegressor(verbosity=0), {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 7]
    }),
    
    # LightGBM
    ("LightGBM", LGBMRegressor(), {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 7]
    })
]

In [ ]:
results=[]
for name, model, param_grid in baseline_models:
    print(f"Training with RandomizedSearch: {name}")

    pipeline = Pipeline(steps=[
        ("preprocess", preprocess),
        ("model", model)
    ])

    # Use RandomizedSearchCV if parameters are provided
    if param_grid:
        random_search = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=param_grid,
            n_iter=10,
            cv=3,
            scoring='neg_mean_squared_error',
            n_jobs=-1,
            verbose=1,
            random_state=42
        )
        random_search.fit(X_train, y_train)
        
        best_pipeline = random_search.best_estimator_
        best_params = random_search.best_params_
        y_pred = best_pipeline.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        
        print(f"{name} -> Best Params: {best_params}")
        print(f"{name} -> Test MSE: {mse:.4f}")
        
        results.append((name, mse, best_pipeline))
        
        # Log experiment with best parameters
        with exp.start_run():
            exp.log_metric("mse", mse)
            exp.log_param("model_type", name)
            for param_name, param_value in best_params.items():
                exp.log_param(param_name, param_value)
            exp.log_model(model=best_pipeline, model_name=f"{name}_model", sample_input_data=X_train.head())
    else:
        # Fit directly without RandomizedSearch for models with no hyperparameters
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        
        results.append((name, mse, pipeline))
        print(f"{name} -> Test MSE: {mse:.4f}")
        
        # Log experiment
        with exp.start_run():
            exp.log_metric("mse", mse)
            exp.log_param("model_type", name)
            exp.log_model(model=pipeline, model_name=f"{name}_model", sample_input_data=X_train.head())

In [ ]:
# Select final features for production based on SHAP importance
num_production_features = 8  # Number of features to select for production

production_features_df = shap_importance_df.copy()
production_features_list = production_features_df.head(num_production_features)['feature'].tolist()

print("=" * 60)
print("RECOMMENDED FEATURES FOR PRODUCTION")
print("=" * 60)
print(f"\nSelected {num_production_features} most important features based on SHAP analysis:")
print()

for i, feature_name in enumerate(production_features_list, 1):
    shap_importance_value = production_features_df[
        production_features_df['feature'] == feature_name
    ]['shap_importance'].values[0]
    print(f"{i}. {feature_name}")
    print(f"   SHAP Importance: {shap_importance_value:.4f}")

print()
print("=" * 60)
print(f"Original number of features: {len(X.columns)}")
print(f"Recommended for production: {len(production_features_list)}")
print(f"Feature reduction: {(1 - len(production_features_list)/len(X.columns))*100:.1f}%")
print("=" * 60)

#### Final Production Feature Set

In [ ]:
# SHAP Waterfall Plot - explains a single prediction
# Shows how each feature contributes to one specific prediction
prediction_index = 0  # First prediction

shap.waterfall_plot(
    shap.Explanation(
        values=shap_values[prediction_index], 
        base_values=shap_explainer.expected_value if hasattr(shap_explainer, 'expected_value') else 0,
        data=X_test_preprocessed[prediction_index],
        feature_names=X_test.columns.tolist()
    )
)

#### SHAP Waterfall Plot (Individual Prediction)

In [ ]:
# SHAP Bar Plot - shows mean absolute SHAP values (global feature importance)
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, 
                  X_test.iloc[:len(shap_values)], 
                  feature_names=X_test.columns.tolist(),
                  plot_type="bar",
                  show=False)
plt.title(f'SHAP Feature Importance - {best_model_name}', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

# Calculate mean absolute SHAP values for each feature
mean_abs_shap_values = np.abs(shap_values).mean(axis=0)
feature_names_list = X_test.columns.tolist()

# Create dataframe with SHAP importance
shap_importance_df = pd.DataFrame({
    'feature': feature_names_list,
    'shap_importance': mean_abs_shap_values
}).sort_values('shap_importance', ascending=False)

print("\nSHAP-based Feature Importance:")
print("=" * 60)
print(shap_importance_df)
print("=" * 60)

#### SHAP Feature Importance

In [ ]:
# SHAP Summary Plot - shows feature importance and impact direction
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, 
                  X_test.iloc[:len(shap_values)], 
                  feature_names=X_test.columns.tolist(),
                  show=False)
plt.title(f'SHAP Summary Plot - {best_model_name}', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

#### SHAP Summary Plot

In [ ]:
import shap

# Prepare data for SHAP analysis
# Transform training and test data using the preprocessing pipeline
X_train_preprocessed = best_model_pipeline.named_steps['preprocess'].transform(X_train)
X_test_preprocessed = best_model_pipeline.named_steps['preprocess'].transform(X_test)

print("=" * 60)
print(f"SHAP ANALYSIS FOR {best_model_name}")
print("=" * 60)

# Create SHAP explainer based on model type
if best_model_name in ['XGBoost', 'LightGBM', 'RandomForest', 'GradientBoosting']:
    # Use TreeExplainer for tree-based models (faster)
    print("Using TreeExplainer for tree-based model...")
    shap_explainer = shap.TreeExplainer(trained_model)
    shap_values = shap_explainer.shap_values(X_test_preprocessed)
    
else:
    # Use KernelExplainer for other models (slower but works for any model)
    print("Using KernelExplainer for linear model...")
    
    # Use smaller sample for faster computation
    sample_size = min(100, len(X_train_preprocessed))
    X_train_sample = X_train_preprocessed[:sample_size]
    
    shap_explainer = shap.KernelExplainer(trained_model.predict, X_train_sample)
    
    # Compute SHAP values for smaller test set
    sample_test_size = min(50, len(X_test_preprocessed))
    shap_values = shap_explainer.shap_values(X_test_preprocessed[:sample_test_size])
    X_test_preprocessed = X_test_preprocessed[:sample_test_size]

print("SHAP values computed successfully!")
print("=" * 60)

#### SHAP Analysis for Best Model

In [ ]:
# Uncomment to install SHAP
# !pip install shap

#### Install SHAP (if needed)

## SHAP Values Analysis

In [ ]:
# Compare feature importance across all tree-based models
all_model_importances = {}

# Extract feature importance from each model
for model_name, model_mse, model_pipeline in results:
    trained_model = model_pipeline.named_steps['model']
    
    if hasattr(trained_model, 'feature_importances_'):
        feature_importance_values = trained_model.feature_importances_
        feature_names_list = X_train.columns.tolist()
        
        importance_df = pd.DataFrame({
            'feature': feature_names_list,
            'importance': feature_importance_values
        }).sort_values('importance', ascending=False)
        
        all_model_importances[model_name] = importance_df

# Plot comparison if we have any tree-based models
if all_model_importances:
    num_models = len(all_model_importances)
    fig, axes = plt.subplots(num_models, 1, figsize=(12, 5 * num_models))
    
    # Handle single model case
    if num_models == 1:
        axes = [axes]
    
    # Plot each model's feature importance
    for i, (model_name, importance_df) in enumerate(all_model_importances.items()):
        top_10_features = importance_df.head(10)
        axes[i].barh(top_10_features['feature'], top_10_features['importance'], color='skyblue')
        axes[i].set_xlabel('Importance')
        axes[i].set_title(f'Top 10 Features - {model_name}')
        axes[i].invert_yaxis()
    
    plt.tight_layout()
    plt.show()
    
    print("Feature importance comparison completed for all tree-based models")
else:
    print("No tree-based models found for feature importance comparison")

#### Feature Importance for All Models

In [ ]:
# Find the best model from results
best_model_name = ""
best_model_mse = float('inf')
best_model_pipeline = None

for model_name, model_mse, model_pipeline in results:
    if model_mse < best_model_mse:
        best_model_mse = model_mse
        best_model_name = model_name
        best_model_pipeline = model_pipeline

print("=" * 60)
print("BEST MODEL")
print("=" * 60)
print(f"Model Name: {best_model_name}")
print(f"Test MSE: {best_model_mse:.4f}")
print("=" * 60)

# Extract the trained model from pipeline
trained_model = best_model_pipeline.named_steps['model']

# Check if model has feature importance
if hasattr(trained_model, 'feature_importances_'):
    # For tree-based models
    feature_importance_values = trained_model.feature_importances_
    feature_names_list = X_train.columns.tolist()
    
    # Create dataframe for feature importance
    importance_dataframe = pd.DataFrame({
        'feature': feature_names_list,
        'importance': feature_importance_values
    }).sort_values('importance', ascending=False)
    
    print("\nFeature Importance Rankings:")
    print(importance_dataframe)
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(importance_dataframe['feature'], importance_dataframe['importance'], color='skyblue')
    plt.xlabel('Importance')
    plt.title(f'Feature Importance - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
    # Select top features for production
    top_n_features = 10
    top_features_list = importance_dataframe.head(top_n_features)['feature'].tolist()
    
    print(f"\nTop {top_n_features} Features Recommended for Production:")
    print("=" * 60)
    for i, feature_name in enumerate(top_features_list, 1):
        importance_value = importance_dataframe[importance_dataframe['feature'] == feature_name]['importance'].values[0]
        print(f"{i}. {feature_name}: {importance_value:.4f}")
    print("=" * 60)

elif hasattr(trained_model, 'coef_'):
    # For linear models
    print(f"\n{best_model_name} is a linear model - using coefficients as importance")
    
    coefficient_values = np.abs(trained_model.coef_)
    feature_names_list = X_train.columns.tolist()
    
    importance_dataframe = pd.DataFrame({
        'feature': feature_names_list,
        'importance': coefficient_values
    }).sort_values('importance', ascending=False)
    
    print("\nFeature Importance (based on absolute coefficients):")
    print(importance_dataframe)
    
    # Plot feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(importance_dataframe['feature'], importance_dataframe['importance'], color='skyblue')
    plt.xlabel('|Coefficient|')
    plt.title(f'Feature Importance - {best_model_name}')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

else:
    print(f"\n{best_model_name} does not support feature importance extraction")

#### Extract Feature Importance from Best Model

## Feature Importance Analysis

## Model registry

In [ ]:
exp.list_artifacts("HOT_KIWI_4", artifact_path="manifest.yaml")

In [ ]:
from snowflake.ml.registry import Registry
registry = Registry(session=session, database_name="dev", schema_name="data_science")

In [ ]:
# # Log the model
# model_version = registry.log_model(
#     model=LinearRegression,
#     model_name="base_linear_regression_model",
#     version_name="v1.0",
#     comment="Base version of the linear regression model",
#     sample_input_data=X.head()
# )
# print(f"Model '{model_version.model_name}' version '{model_version.version_name}' registered successfully.")

## Inference